In [8]:
%load_ext autoreload
%autoreload 2

import os,sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath("../training"))
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os
os.environ["EXP_NAME"] = "movie_recom"
os.environ["MAX_RATING"] = "5"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:8081"
os.environ["EPOCH"] = "2"

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [10]:
from training.xgb import main, test_xgb_model
from movie_recommender.workflow import read_parquet_from_s3, connect_minio,connect_mlflow

DB_MINIO_BUCKET = "trainingbucket"
connect_minio()
connect_mlflow()

[DEBUG] - Available Buckets [Bucket('mlflowbucket'), Bucket('trainingbucket')]


In [11]:
ratings = read_parquet_from_s3(DB_MINIO_BUCKET, "ratings.parquet")
movies = read_parquet_from_s3(DB_MINIO_BUCKET, "movies.parquet")
ratings = ratings.merge(movies, on="movie_id")
ratings.rating *= 5

In [12]:
booster = main(ratings,n_trials=1)

[INFO] - Running CV...
[I 2025-09-22 09:47:31,237] A new study created in memory with name: no-name-37ca6c18-1d04-49b3-8833-0e65e2f2aba0


  0%|          | 0/1 [00:00<?, ?it/s]

[DEBUG] - Pred mean: 0.71 , std: 0.13
[DEBUG] - Pred mean: 0.71 , std: 0.13
[DEBUG] - Pred mean: 0.71 , std: 0.13
[INFO] - Found best params: {'objective': 'reg:squarederror', 'device': 'cuda', 'random_state': 0, 'eval_metric': 'rmse', 'booster': 'gbtree', 'max_depth': 7, 'learning_rate': 0.20063357546237642, 'subsample': 0.9544887212624263, 'colsample_bytree': 0.5457841395515235, 'gamma': 3.748739870376525, 'lambda': 0.004309860156011425, 'alpha': 0.008046848192811441}


[I 2025-09-22 09:48:37,222] Trial 0 finished with value: 0.6340526938438416 and parameters: {'objective': 'reg:squarederror', 'device': 'cuda', 'random_state': 0, 'eval_metric': 'rmse', 'booster': 'gbtree', 'max_depth': 7, 'learning_rate': 0.20063357546237642, 'subsample': 0.9544887212624263, 'colsample_bytree': 0.5457841395515235, 'gamma': 3.748739870376525, 'lambda': 0.004309860156011425, 'alpha': 0.008046848192811441}. Best is trial 0 with value: 0.6340526938438416.
[0]	train-rmse:0.19618	val-rmse:0.19773
[530]	train-rmse:0.15734	val-rmse:0.16982


[INFO] - Found most impactful features: ['movie_avg_rating', 'mean_Drama', 'mean_Sci-Fi', 'mean_Thriller', 'mean_Action', 'mean_Comedy', 'mean_Fantasy', 'mean_Adventure', 'user_avg_rating']
[INFO] - 

[INFO] - Training XGBMR...
[INFO] - Run id: 62c5c84fa49f4016a7117210548d605a


[0]	train-rmse:0.19683	train-mae:0.75982	val-rmse:0.19841	val-mae:0.76457
[500]	train-rmse:0.15908	train-mae:0.59110	val-rmse:0.17108	val-mae:0.63452
[1000]	train-rmse:0.15907	train-mae:0.59100	val-rmse:0.17109	val-mae:0.63445
[1078]	train-rmse:0.15907	train-mae:0.59099	val-rmse:0.17109	val-mae:0.63445


2025/09/22 09:49:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
[INFO] - Logging artifacts.
[DEBUG] - Logging folder %s
[INFO] - Done training.


🏃 View run persistent-eel-5 at: http://localhost:8081/#/experiments/1/runs/62c5c84fa49f4016a7117210548d605a
🧪 View experiment at: http://localhost:8081/#/experiments/1


Registered model 'xgb' already exists. Creating a new version of this model...
2025/09/22 09:49:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb, version 3
Created version '3' of model 'xgb'.
[DEBUG] - Champion 'val-mae': 0.6314, Latest 'val-mae': 0.6344
[DEBUG] - No new champion for 'xgb'
[INFO] - 

[INFO] - Saving plots...
[INFO] - True  : mean 3.557, std 1.051
[INFO] - Pred  : mean 3.557, std 0.673
[INFO] - MAE   : 0.5905273019984898
[INFO] - Error : mean 0.000, std 0.795
[INFO] - Test
[INFO] - True  : mean 3.557, std 1.049
[INFO] - Pred  : mean 3.555, std 0.663
[INFO] - MAE   : 0.6341939829993853
[INFO] - Error : mean 0.002, std 0.855
[DEBUG] - Logging folder %s


In [13]:
test_xgb_model()

[INFO] - XGB loaded on device: cuda
[DEBUG] - Getting artifacts, run_id='62c5c84fa49f4016a7117210548d605a', artifact_path='resources'


[INFO] - Loaded champion model, registered_name='xgb'
[INFO] - XGB model test passed successfully
